# Which COVID-19 Vaccine is Best for You?
## DS4A Team 55 Preliminary EDA


In [1]:
import pandas as pd
import numpy as np
import math
import base64
import matplotlib.pyplot as plt
import seaborn as sns
from sodapy import Socrata
import zipfile

In [2]:
# increase the size and quality of any plots we produce
import matplotlib as mpl
mpl.rcParams["figure.dpi"] = 300

## state_vax.csv
Vaccine administration data for states within the U.S., including the District of Columbia from 12/13/2020 to the present, updated daily. 

In [3]:
client = Socrata(
        domain = "data.cdc.gov",
        app_token = "bsYMLkKAQhIIVd7wzBzp5BiCR",
        timeout=10
        )
results = client.get_all("unsk-b7fc")

# convert to pandas DataFrame
df = pd.DataFrame.from_records(results)
df.sample(10)

,date,mmwr_week,location,distributed,distributed_janssen,distributed_moderna,distributed_pfizer,distributed_unk_manuf,dist_per_100k,distributed_per_100k_12plus,...,administered_dose1_recip_5pluspop_pct,series_complete_5plus,series_complete_5pluspop_pct,administered_5plus,admin_per_100k_5plus,distributed_per_100k_5plus,series_complete_moderna_5plus,series_complete_pfizer_5plus,series_complete_janssen_5plus,series_complete_unk_manuf_5plus
9141,2021-09-14T00:00:00.000,37,LA,5470210,238100,2338100,2894010,0,117669,139428,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
1433,2022-01-12T00:00:00.000,2,MH,68980,12100,47700,9180,0,88731,120239,...,38.1,23113,33.4,48965,70761,99685,19007,1530,2575,1
23359,2021-02-08T00:00:00.000,6,MT,163675,0,0,0,0,15314,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
7958,2021-10-02T00:00:00.000,39,TX,42069505,2031100,16295760,23742645,0,145088,174554,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
26042,2020-12-28T00:00:00.000,53,UT,104925,0,0,0,0,3273,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
21021,2021-03-15T00:00:00.000,11,AR,1251620,33000,638300,580320,0,41475,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
10461,2021-08-25T00:00:00.000,34,KS,3507375,173500,1469600,1864275,0,120391,143059,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
3817,2021-12-06T00:00:00.000,49,IN,10246880,545800,3692720,6008360,0,152207,179611,...,60,3431397,54.3,7952255,125949,162291,1208162,1948415,267088,7732
21384,2021-03-10T00:00:00.000,10,OH,4444875,133000,2286800,2025075,0,38026,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
9490,2021-09-09T00:00:00.000,36,MO,7509375,311400,3007260,4190715,0,122354,143426,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN


In [4]:
df.columns

Index(['date', 'mmwr_week', 'location', 'distributed', 'distributed_janssen',
       'distributed_moderna', 'distributed_pfizer', 'distributed_unk_manuf',
       'dist_per_100k', 'distributed_per_100k_12plus',
       'distributed_per_100k_18plus', 'distributed_per_100k_65plus',
       'administered', 'administered_12plus', 'administered_18plus',
       'administered_65plus', 'administered_janssen', 'administered_moderna',
       'administered_pfizer', 'administered_unk_manuf', 'admin_per_100k',
       'admin_per_100k_12plus', 'admin_per_100k_18plus',
       'admin_per_100k_65plus', 'recip_administered',
       'administered_dose1_recip', 'administered_dose1_pop_pct',
       'administered_dose1_recip_1', 'administered_dose1_recip_2',
       'administered_dose1_recip_3', 'administered_dose1_recip_4',
       'administered_dose1_recip_5', 'administered_dose1_recip_6',
       'series_complete_yes', 'series_complete_pop_pct',
       'series_complete_12plus', 'series_complete_12pluspop',
    

In [5]:
df.sort_values(by="date", ascending=True)

,date,mmwr_week,location,distributed,distributed_janssen,distributed_moderna,distributed_pfizer,distributed_unk_manuf,dist_per_100k,distributed_per_100k_12plus,...,administered_dose1_recip_5pluspop_pct,series_complete_5plus,series_complete_5pluspop_pct,administered_5plus,admin_per_100k_5plus,distributed_per_100k_5plus,series_complete_moderna_5plus,series_complete_pfizer_5plus,series_complete_janssen_5plus,series_complete_unk_manuf_5plus
26967,2020-12-13T00:00:00.000,51,LTC,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
26962,2020-12-13T00:00:00.000,51,US,13650,0,0,0,0,4,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
26963,2020-12-13T00:00:00.000,51,GU,3900,0,0,0,0,2353,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
26966,2020-12-13T00:00:00.000,51,VI,975,0,0,0,0,931,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
26965,2020-12-13T00:00:00.000,51,AS,3900,0,0,0,0,7003,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2022-02-03T00:00:00.000,5,PR,7125970,213300,2505640,4407030,0,223126,250184,...,95,2545210,82.7,6573558,213690,231648,940104,1466265,138711,130
35,2022-02-03T00:00:00.000,5,AS,91010,600,21500,68910,0,192037,246526,...,95,37547,87.1,84307,195563,211111,9001,27970,567,9
34,2022-02-03T00:00:00.000,5,OH,21358395,965100,7941580,12451715,0,182721,213569,...,66.3,6653270,60.5,16960514,154211,194198,2350721,3753677,543725,5147
32,2022-02-03T00:00:00.000,5,NV,5505590,257000,1869980,3378610,0,178744,209993,...,77.8,1806176,62.4,4604387,159069,190203,591684,1044495,169942,55


In [6]:
# df.shape

In [7]:
# remove all distribution cols
# remove "Administered", this is the total for 1st dose and complete series
# remove all 5plus cols, very recently approved and sporadic across states
# remove additional doses for 50plus, age group doesn't show up elsewhere in dataset


df = df.drop(columns= ["mmwr_week","distributed", "distributed_janssen", "distributed_moderna", 
                       "distributed_pfizer", "distributed_unk_manuf", "dist_per_100k","distributed_per_100k_12plus",
                       "distributed_per_100k_18plus", "distributed_per_100k_65plus", "distributed_per_100k_5plus",
                       "administered","administered_dose1_recip_5plus", "recip_administered",
                       "administered_dose1_recip_5pluspop_pct", "series_complete_5plus", "series_complete_5pluspop_pct", 
                       "administered_5plus", "admin_per_100k_5plus", "series_complete_moderna_5plus",
                       "series_complete_pfizer_5plus", "series_complete_janssen_5plus", "series_complete_unk_manuf_5plus",
                       "additional_doses_50plus", "additional_doses_50plus_vax_pct"], axis=1)

In [8]:
# abbreviating column names for readability
# cols in api data are labelled differently than csv directly from data.cdc.gov, values match up so renaming those cols

rename_cols = {"administered_12plus":"admin_12plus",
               "location":"state",
               "administered_18plus":"admin_18plus",
               "administered_65plus":"admin_65plus",
               "administered_janssen":"admin_janssen",
               "administered_moderna":"admin_moderna",
               "administered_pfizer":"admin_pfizer",
               "administered_unk_manuf":"admin_unk",
               "administered_dose1_recip":"admin_dose1",
               "administered_dose1_pop_pct":"admin_dose1_pop_pct",
               "administered_dose1_recip_1":"admin_dose1_12plus",
               "administered_dose1_recip_2":"admin_dose1_12plus_pop_pct",
               "administered_dose1_recip_3":"admin_dose1_18plus",
               "administered_dose1_recip_4":"admin_dose1_18plus_pop_pct",
               "administered_dose1_recip_5":"admin_dose1_65plus",
               "administered_dose1_recip_6":"admin_dose1_65plus_pop_pct",
               "series_complete_yes":"series_complete",
               "series_complete_12pluspop":"series_complete_12plus_pop_pct",
               "series_complete_18pluspop":"series_complete_18plus_pop_pct",
               "series_complete_65pluspop":"series_complete_65plus_pop_pct",
               "series_complete_unk_manuf":"series_complete_unk",
               "series_complete_unk_manuf_1":"series_complete_unk_12plus",
               "series_complete_unk_manuf_2":"series_complete_unk_18plus",
               "series_complete_unk_manuf_3":"series_complete_unk_65plus",
               "series_complete_12pluspop_pct":"series_complete_12plus_pop_pct",
               "series_complete_18pluspop_pct":"series_complete_18plus_pop_pct",
               "series_complete_65pluspop_pct":"series_complete_65plus_pop_pct"
               }
df = df.rename(columns=rename_cols)

In [9]:
# df.columns

In [10]:
# see data types of df
# all types = object, need to change to ints and floats
# df.dtypes

In [11]:
# show cols with NaNs
# cols with NaNs are additional dose cols, after investigating this happens when other additional dose cols are 0
df.columns[df.isnull().any()]

Index(['additional_doses', 'additional_doses_vax_pct',
       'additional_doses_18plus', 'additional_doses_18plus_vax_pct',
       'additional_doses_65plus', 'additional_doses_65plus_vax_pct',
       'additional_doses_moderna', 'additional_doses_pfizer',
       'additional_doses_janssen', 'additional_doses_unk_manuf'],
      dtype='object')

In [12]:
# replace NaNs with 0 to fill in values
nan_col = ['additional_doses', 'additional_doses_vax_pct',
           'additional_doses_18plus', 'additional_doses_18plus_vax_pct',
           'additional_doses_65plus', 'additional_doses_65plus_vax_pct',
           'additional_doses_moderna', 'additional_doses_pfizer',
           'additional_doses_janssen', 'additional_doses_unk_manuf']
df[nan_col] = df[nan_col].fillna(0)

In [13]:
to_int = ['admin_12plus', 'admin_18plus', 'admin_65plus','admin_janssen',
          'admin_moderna', 'admin_pfizer', 'admin_unk','admin_per_100k',
          'admin_per_100k_12plus', 'admin_per_100k_18plus','admin_per_100k_65plus',
          'admin_dose1', 'admin_dose1_12plus', 'admin_dose1_18plus',
          'admin_dose1_65plus', 'series_complete_12plus', 'series_complete_18plus',
          'series_complete','series_complete_65plus', 'series_complete_janssen', 'series_complete_moderna',
          'series_complete_pfizer','series_complete_unk', 'series_complete_janssen_12plus',
          'series_complete_moderna_12plus', 'series_complete_pfizer_12plus', 'series_complete_unk_12plus',
          'series_complete_janssen_18plus', 'series_complete_moderna_18plus', 'series_complete_pfizer_18plus',
          'series_complete_unk_18plus', 'series_complete_janssen_65plus', 'series_complete_moderna_65plus',
          'series_complete_pfizer_65plus', 'series_complete_unk_65plus', 'additional_doses',
          'additional_doses_18plus', 'additional_doses_65plus', 'additional_doses_moderna',
          'additional_doses_pfizer', 'additional_doses_janssen', 'additional_doses_unk_manuf'
         ]

to_float = ['admin_dose1_pop_pct', 'admin_dose1_12plus_pop_pct', 'admin_dose1_18plus_pop_pct',
            'admin_dose1_65plus_pop_pct', 'series_complete_pop_pct', 'series_complete_12plus_pop_pct',
            'series_complete_12plus_pop_pct', 'series_complete_18plus_pop_pct', 'series_complete_65plus_pop_pct',
            'additional_doses_vax_pct', 'additional_doses_18plus_vax_pct', 'additional_doses_65plus_vax_pct'
           ]

df[to_int] = df[to_int].astype("int")
df[to_float] = df[to_float].astype("float")

In [14]:
# checking that data types hace been converted correctly
# df.dtypes

In [15]:
# removing locations outside of the contential united states as well as united states total

drop_location = ["AS", "BP2", "DD2", "FM", "GU",
                 "IH2", "MH", "MP", "PR", "RP",
                 "US", "VA2", "VI", "LTC"]
df = df[~df["state"].isin(drop_location)]

# checking that location vals = 51, including DC

df.state.nunique()

52

In [16]:
# converting "date" col to datetime
df["date"] = pd.to_datetime(df["date"])

In [17]:
# filter location to New York
df_ny = df.loc[df["state"] == "NY"]

In [18]:
# creating a new df with all values on a daily basis instead of a running total
df_ny_daily = df_ny.drop(columns="state").set_index("date")
df_ny_daily = df_ny_daily.diff(periods=-1)
df_ny_daily.head(50)

,admin_12plus,admin_18plus,admin_65plus,admin_janssen,admin_moderna,admin_pfizer,admin_unk,admin_per_100k,admin_per_100k_12plus,admin_per_100k_18plus,...,additional_doses,additional_doses_vax_pct,additional_doses_18plus,additional_doses_18plus_vax_pct,additional_doses_65plus,additional_doses_65plus_vax_pct,additional_doses_moderna,additional_doses_pfizer,additional_doses_janssen,additional_doses_unk_manuf
date,,,,,,,,,,,,,,,,,,,,,
2022-02-03,43152.0,38981.0,5802.0,1176.0,15193.0,33279.0,94.0,255.0,258.0,253.0,...,22990.0,0.1,21168.0,0.1,3840.0,0.1,9048.0,13455.0,486.0,1.0
2022-02-02,49261.0,43520.0,5749.0,1120.0,15910.0,39955.0,65.0,294.0,293.0,282.0,...,26459.0,0.2,23627.0,0.2,3802.0,0.1,9491.0,16444.0,522.0,2.0
2022-02-01,32802.0,29186.0,3318.0,783.0,10536.0,25578.0,66.0,190.0,196.0,189.0,...,17349.0,0.1,15397.0,0.0,2117.0,0.1,6179.0,10827.0,342.0,1.0
2022-01-31,17424.0,14387.0,1130.0,377.0,5203.0,14899.0,10.0,105.0,104.0,94.0,...,10450.0,0.0,8462.0,0.1,805.0,0.0,3429.0,6872.0,149.0,0.0
2022-01-30,49846.0,42207.0,4525.0,888.0,16437.0,40187.0,24.0,296.0,297.0,273.0,...,32199.0,0.2,27294.0,0.2,3249.0,0.1,11450.0,20281.0,468.0,0.0
2022-01-29,61485.0,55269.0,7659.0,1403.0,20333.0,47749.0,117.0,357.0,366.0,359.0,...,35979.0,0.2,32669.0,0.2,5271.0,0.2,13211.0,22050.0,712.0,6.0
2022-01-28,58849.0,53267.0,7770.0,1242.0,20191.0,45159.0,90.0,343.0,351.0,345.0,...,33572.0,0.2,30765.0,0.2,5327.0,0.2,12986.0,20009.0,569.0,8.0
2022-01-27,56316.0,50669.0,7246.0,1249.0,18474.0,44217.0,102.0,329.0,336.0,328.0,...,31769.0,0.2,28954.0,0.2,4967.0,0.1,12033.0,19226.0,507.0,3.0
2022-01-26,66938.0,59658.0,8305.0,1499.0,22891.0,50887.0,71.0,388.0,399.0,387.0,...,38505.0,0.2,34484.0,0.2,5864.0,0.2,14411.0,23470.0,620.0,4.0


In [19]:
# this value appears as an outlier but was manually calculated from raw dataset and is correct
print(df_ny_daily["admin_pfizer"].max())

581745.0


In [20]:
# set index to date to extract rows for the first of each month, aligns with how other datasets are reported
df_ny_m = df_ny.set_index("date")
df_ny_m = df_ny_m.groupby(df_ny_m.index.strftime("%Y-%m")).tail(1)
df_ny_m = df_ny_m.reset_index().sort_values(by="date", ascending=True)
# df_ny_m.head(10)

In [21]:
# first reported date and most recent date for vax data, updated daily
print(df_ny["date"].min())
print(df_ny["date"].max())

2020-12-14 00:00:00
2022-02-03 00:00:00


# covid_cases
Covid cases in NY

In [22]:
# #read data
# covid_df = pd.read_csv("data/covid_cases_4states.csv.zip", compression="zip")
# covid_df.head()

with zipfile.ZipFile("data/covid_cases_4states.csv.zip") as z:
    with z.open("covid_cases_4states.csv") as f:
        covid_df = pd.read_csv(f, low_memory=False)

covid_df.head()
# # open zipped dataset
# with zipfile.ZipFile("test.zip") as z:
#    # open the csv file in the dataset
#    with z.open("test.csv") as f:
       
#       # read the dataset
#       train = pd.read_csv(f)

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,2021-10,NY,36,CLINTON,36019.0,65+ years,Female,Missing,NaN,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,NaN
1,2021-07,FL,12,BAKER,12003.0,50 to 64 years,Female,NaN,NaN,NaN,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,No,Missing,No,NaN
2,2020-09,NY,36,STEUBEN,36101.0,50 to 64 years,Female,Missing,NaN,0.0,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,NaN
3,2021-02,FL,12,MANATEE,12081.0,18 to 49 years,Female,Unknown,Hispanic/Latino,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
4,2021-03,FL,12,MARION,12083.0,50 to 64 years,Female,Unknown,Hispanic/Latino,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN


In [23]:
covid_df.columns #column names

Index(['case_month', 'res_state', 'state_fips_code', 'res_county',
       'county_fips_code', 'age_group', 'sex', 'race', 'ethnicity',
       'case_positive_specimen_interval', 'case_onset_interval', 'process',
       'exposure_yn', 'current_status', 'symptom_status', 'hosp_yn', 'icu_yn',
       'death_yn', 'underlying_conditions_yn'],
      dtype='object')

In [24]:
# covid_df.dtypes

In [25]:
covid_df.shape

(11685325, 19)

In [26]:
#data cleaning
covid_df['case_month'] = pd.to_datetime(covid_df['case_month']) #convert to date

In [27]:
# dropping irrelevant columns
covid_df = covid_df.drop(columns=["case_positive_specimen_interval", "case_onset_interval", "process", "exposure_yn", "current_status"])

In [28]:
# rename cols for readability and uniformify across datasets
covid_rename = {"case_month":"date", "res_state":"state",
               "res_county":"county"}

covid_df = covid_df.rename(columns=covid_rename)

In [29]:
# converting data types 
to_str = ["state", "county", "age_group", "sex", "race", "ethnicity",
          "symptom_status", "hosp_yn", "icu_yn", "death_yn", "underlying_conditions_yn"]
covid_df[to_str] = covid_df[to_str].astype(str)

In [30]:
# show cols with NaNs
covid_df.columns[covid_df.isnull().any()]

Index(['county_fips_code'], dtype='object')

In [31]:
# fill NaNs in county_fips_code with "Not Available"
covid_df["county_fips_code"] = covid_df["county_fips_code"].fillna("Not Available")

# shows all values within the race col
covid_df.race.unique()

# rename nan, Missing, and Unknown to Not Available
rename_covid_nan = {"nan":"Not Available",
                     "Missing":"Not Available",
                     "Unknown":"Not Available"}
covid_df["race"] = covid_df["race"].replace(rename_covid_nan).astype(str)

# same treatment for ethnicity as the race col
covid_df.ethnicity.unique()

covid_df["ethnicity"] = covid_df["ethnicity"].replace(rename_covid_nan).astype(str)

covid_df["county"] = covid_df["county"].replace({"nan":"Not Available"})

# checking for NaN string values and replace with "Not Available" in remaining cols
covid_df.age_group.unique()
covid_df["age_group"] = covid_df["age_group"].replace({"nan":"Not Available", "Missing":"Not Available"}).astype(str)

covid_df["sex"] = covid_df["sex"].replace(rename_covid_nan).astype(str)

covid_df["death_yn"] = covid_df["death_yn"].replace(rename_covid_nan).astype(str)

covid_df["underlying_conditions_yn"] = covid_df["underlying_conditions_yn"].replace(rename_covid_nan).astype(str)

rename_missing_unk = {"Missing":"Not Available",
                      "Unknown":"Not Available"}
rename_col = ["symptom_status", "hosp_yn", "icu_yn"]
covid_df[rename_col] = covid_df[rename_col].replace(rename_missing_unk).astype(str)

In [32]:
covid_df.sample(10, random_state = 42)

,date,state,state_fips_code,county,county_fips_code,age_group,sex,race,ethnicity,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
11639060,2021-12-01,TX,48,Not Available,Not Available,18 to 49 years,Male,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
7365273,2021-01-01,CA,6,KERN,6029.0,18 to 49 years,Female,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
2826409,2020-05-01,CA,6,MONTEREY,6053.0,18 to 49 years,Male,White,Hispanic/Latino,Symptomatic,No,Not Available,Not Available,Not Available
3894756,2021-08-01,NY,36,ORANGE,36071.0,18 to 49 years,Female,Not Available,Hispanic/Latino,Symptomatic,No,Not Available,No,Not Available
8602615,2021-08-01,NY,36,SUFFOLK,36103.0,65+ years,Male,Not Available,Not Available,Not Available,Not Available,Not Available,No,Not Available
10347080,2021-09-01,FL,12,MIAMI-DADE,12086.0,65+ years,Female,White,Hispanic/Latino,Not Available,No,Not Available,No,Not Available
5423655,2021-09-01,NY,36,MONROE,36055.0,0 - 17 years,Female,Black,Non-Hispanic/Latino,Not Available,Not Available,Not Available,No,Not Available
5897489,2020-12-01,CA,6,ALAMEDA,6001.0,18 to 49 years,Male,Multiple/Other,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
9556751,2021-01-01,NY,36,QUEENS,36081.0,18 to 49 years,Female,White,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
1140622,2021-09-01,NY,36,CHAUTAUQUA,36013.0,18 to 49 years,Female,Not Available,Not Available,Not Available,Not Available,Not Available,No,Not Available


In [33]:
# covid_df.describe(include = 'all',datetime_is_numeric=True) #describe and include string columns

In [34]:
#covid cases trend
# trend_df = covid_df.groupby(['date']).size().reset_index(name = 'case_count').set_index('date')
# trend_df['case_count_per_100k'] = trend_df['case_count']/100000
# plt.plot(trend_df.index,trend_df['case_count_per_100k'])
# plt.title('COVID-19 Cases Have Spiked Twice And Is Trending Upwards')
# plt.xlabel('Date')
# plt.ylabel('Case Count (Hundred Thousand)')
# plt.xticks(rotation=90)
# plt.show()

In [35]:
# trend_df.head()

In [36]:
# comparing cases with vaccine administration rates
# case_vax_ax = ny_m_plt.plot(x="date", y=["admin_per_100k_12plus", "admin_per_100k_18plus", "admin_per_100k_65plus"], kind="line")
# trend_df.plot(y="case_count", kind="line", ax=case_vax_ax, title="COVID-19 Cases and Vaccinations per Age Group");                            

When comparing the COVID case trends with the trends in vaccine administration across each age group, it can be seen that the number of cases starts to sharply decline as the population recieves the vaccine. The case_count trend line was already falling when the vaccines were made available. However, once more of the population started to get vaccinated the case numbers stayed low up until the omicron variant began spreading around November of 2021 up until now (mid-January 2022).

In [37]:
# case_comp_ax = daily_plt.plot(x="date", y='series_complete', kind="line")
# trend_df.plot(y="case_count", kind="line", ax=case_comp_ax, title="COVID-19 Cases and Complete Vaccine Series");

In [38]:
# covid_65plus = covid_df.loc[covid_df["age_group"] == "65+ years"]


In [39]:
# df_covid65 = covid_65plus.groupby(['date']).size().reset_index(name = 'case_count').set_index('date')

# df_covid65.plot(title="Trend in COVID-19 Cases for Age 65+");

In [40]:
# covid65_ax = ny_m_plt.plot(x="date", y="admin_per_100k_65plus", kind="line")
# df_covid65.plot(y="case_count", kind="line", ax=covid65_ax, title="COVID-19 Cases and Vaccinations for Age 65+");

# Analyzing the Impact of Different Factors on Case Trends
### Applying multiple regression to a new data frame containing case counts for each state by month, vaccine administration per month for each manufacturer, and analyzing demographic factors

In [41]:
# cases per state

cases_4state = covid_df.groupby(["date", "state"], as_index=False).size()
cases_4state = cases_4state.rename(columns={"size":"cases"})
cases_4state = cases_4state[~(cases_4state["date"] <= "2020-12-01")]


In [42]:
cases_4state

,date,state,cases
48,2021-01-01,CA,877185
49,2021-01-01,FL,207612
50,2021-01-01,NY,333399
51,2021-01-01,TX,5057
52,2021-02-01,CA,182380
53,2021-02-01,FL,165000
54,2021-02-01,NY,199327
55,2021-02-01,TX,2108
56,2021-03-01,CA,83365
57,2021-03-01,FL,330204


In [43]:
keep_state = ["NY", "CA", "FL", "TX"]
df_4state = df.loc[df["state"].isin(keep_state)]


# df_4state_m = df_4state.set_index("date")
# df_4state_m = df_4state_m.groupby(df_4state_m.index.strftime("%Y-%m")).tail(1)
# df_4state_m = df_4state_m.reset_index().sort_values(by="date", ascending=True)
# manuf_4state_m = df_4state_m[["date","state", "admin_janssen", "admin_moderna", "admin_pfizer", "admin_unk"]]

In [44]:
df_4state_m = df_4state[df_4state["date"].dt.is_month_start].sort_values(by="date", ascending=True)
manuf_4state_m = df_4state_m[["date","state", "admin_janssen", "admin_moderna", "admin_pfizer", "admin_unk"]]


In [45]:
manuf_4state_m

,date,state,admin_janssen,admin_moderna,admin_pfizer,admin_unk
25784,2021-01-01,TX,0,221764,301502,0
25775,2021-01-01,FL,0,104492,133296,21
25780,2021-01-01,CA,0,60934,313563,0
25778,2021-01-01,NY,0,82248,146240,11
23799,2021-02-01,TX,0,1297536,1198696,0
23787,2021-02-01,FL,0,971820,1032236,1322
23755,2021-02-01,CA,0,1600685,1852599,0
23752,2021-02-01,NY,0,943356,984098,211
21990,2021-03-01,CA,352,4460552,4645163,10
21972,2021-03-01,FL,0,2509245,2533476,5239


In [56]:
covid_4state = pd.merge(case_manuf, manuf_4state_m, on=["date", "state"], how="inner")

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [57]:
covid_4state

,date,state,cases,admin_janssen,admin_moderna,admin_pfizer,admin_unk
0,2021-01-01,CA,877185,0,60934,313563,0
1,2021-01-01,FL,207612,0,104492,133296,21
2,2021-01-01,NY,333399,0,82248,146240,11
3,2021-01-01,TX,5057,0,221764,301502,0
4,2021-02-01,CA,182380,0,1600685,1852599,0
5,2021-02-01,FL,165000,0,971820,1032236,1322
6,2021-02-01,NY,199327,0,943356,984098,211
7,2021-02-01,TX,2108,0,1297536,1198696,0
8,2021-03-01,CA,83365,352,4460552,4645163,10
9,2021-03-01,FL,330204,0,2509245,2533476,5239


In [48]:
# df_ny_daily = df_ny.drop(columns="state").set_index("date")
# df_ny_daily = df_ny_daily.diff(periods=-1)

In [49]:
manuf_4state_m.dtypes

date             datetime64[ns]
state                    object
admin_janssen             int64
admin_moderna             int64
admin_pfizer              int64
admin_unk                 int64
dtype: object

In [50]:
case_manuf = manuf_4state_m.sort_values(["date", "state"])
case_manuf = case_manuf.join(case_manuf.groupby("state")[["admin_janssen", "admin_moderna", "admin_pfizer", "admin_unk"]].diff().fillna(0), rsuffix="_diff")

In [51]:
case_manuf = case_manuf.drop(columns=["admin_janssen", "admin_moderna", "admin_pfizer", "admin_unk"])

In [52]:
# converting state names to numerics to fit regression line
# CA:1, FL:2, NY:3, TX:4

numstate = {"CA":1, "FL":2, "NY":3, "TX":4}
case_manuf["state"] = case_manuf["state"].replace(numstate)

In [55]:
case_manuf

,date,state,admin_janssen_diff,admin_moderna_diff,admin_pfizer_diff,admin_unk_diff
25780,2021-01-01,1,0.0,0.0,0.0,0.0
25775,2021-01-01,2,0.0,0.0,0.0,0.0
25778,2021-01-01,3,0.0,0.0,0.0,0.0
25784,2021-01-01,4,0.0,0.0,0.0,0.0
23755,2021-02-01,1,0.0,1539751.0,1539036.0,0.0
23787,2021-02-01,2,0.0,867328.0,898940.0,1301.0
23752,2021-02-01,3,0.0,861108.0,837858.0,200.0
23799,2021-02-01,4,0.0,1075772.0,897194.0,0.0
21990,2021-03-01,1,352.0,2859867.0,2792564.0,10.0
21972,2021-03-01,2,0.0,1537425.0,1501240.0,3917.0


In [53]:
case_manuf.describe(include = 'all', datetime_is_numeric=True)

,date,state,admin_janssen_diff,admin_moderna_diff,admin_pfizer_diff,admin_unk_diff
count,56,56.000000,56.000000,5.600000e+01,5.600000e+01,56.000000
mean,2021-07-17 01:42:51.428571392,2.500000,117016.732143,1.217038e+06,1.936151e+06,2911.196429
min,2021-01-01 00:00:00,1.000000,0.000000,0.000000e+00,0.000000e+00,-1485.000000
25%,2021-04-01 00:00:00,1.750000,36431.750000,5.652140e+05,1.219378e+06,187.500000
50%,2021-07-16 12:00:00,2.500000,75343.500000,9.475955e+05,1.562544e+06,1173.500000
75%,2021-11-01 00:00:00,3.250000,153654.500000,1.538006e+06,2.457557e+06,2513.500000
max,2022-02-01 00:00:00,4.000000,731072.000000,4.850227e+06,6.633513e+06,16986.000000
std,NaN,1.128152,133971.429242,1.001521e+06,1.250359e+06,4383.984997


In [54]:
case_manuf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 25780 to 144
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                56 non-null     datetime64[ns]
 1   state               56 non-null     int64         
 2   admin_janssen_diff  56 non-null     float64       
 3   admin_moderna_diff  56 non-null     float64       
 4   admin_pfizer_diff   56 non-null     float64       
 5   admin_unk_diff      56 non-null     float64       
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 5.1 KB
